### PCA y KMeans en minería de datos: un estudio de caso con listados de Airbnb

#### Introducción a la PCA
El análisis de componentes principales (PCA) es una técnica estadística que simplifica la complejidad de los datos de alta dimensión conservando tendencias y patrones. Lo hace transformando los datos en menos dimensiones, que actúan como resúmenes de características, llamados componentes principales (PC). Estos componentes son ortogonales entre sí, lo que garantiza que representen variaciones independientes en los datos.



#### Descripción general del conjunto de datos
El caso de estudio , utiliza un conjunto de datos de listados de Airbnb que contiene varias características como ubicación, tipo de habitación, precio y más. Nuestro objetivo es descubrir patrones subyacentes en este conjunto de datos que puedan ayudarnos a segmentar los listados en grupos significativos.

In [20]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [21]:
# Load the dataframe from the CSV file
df = pd.read_csv('https://github.com/Wilsonsr/Metodos-Estadisticos/raw/main/BASES/airbnb.csv')
df

,listing_id,host_id,host_since_datekey,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,neighbourhood,city,latitude,longitude,property_type,accommodates,bedrooms,price,review_scores_rating,instant_bookable,listing_size_sqft
0,46096773,349135088,20200608,NaN,0.26,f,5.0,Enclos-St-Laurent,Paris,48.879289,2.354539,Room in hotel,0,NaN,0.0,18,f,130.565530
1,44692017,361914459,20200812,NaN,0.34,f,0.0,Palais-Bourbon,Paris,48.854270,2.307810,Room in boutique hotel,0,NaN,0.0,88,f,286.832585
2,45841514,371644174,20201012,NaN,0.51,f,0.0,Murray Hill,New York,40.750920,-73.980050,Room in boutique hotel,0,NaN,0.0,75,f,294.833871
3,42228997,314197504,20191203,NaN,0.33,f,0.0,Lower East Side,New York,40.721860,-73.992780,Room in hotel,4,NaN,0.0,13,f,403.228927
4,48115028,387570084,20200429,NaN,0.64,f,0.0,Murray Hill,New York,40.750920,-73.980050,Room in boutique hotel,0,NaN,0.0,92,f,342.782170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7205,48109312,387561857,20200526,1.0,0.47,f,0.0,Fremont,Sydney,-33.889040,151.270220,NaN,6,3.0,NaN,96,f,NaN
7206,6569075,235767408,20190110,NaN,0.61,f,19.0,Fremont,Sydney,-33.893720,151.216610,NaN,2,1.0,NaN,96,t,NaN
7207,7736378,295593456,20190917,1.0,0.34,t,2.0,Fremont,Sydney,-33.880730,151.213870,NaN,2,1.0,NaN,96,t,NaN
7208,7699700,293573248,20190910,1.0,0.91,f,0.0,Fremont,Sydney,-33.875050,151.221260,NaN,2,1.0,NaN,98,t,NaN


- **listing_id:** Identificador único para cada anuncio en la plataforma de Airbnb.
- **host_id:** Identificador único para cada anfitrión en la plataforma.|
- **host_since_datekey:** Fecha en la que el anfitrión se unió a Airbnb.
- **host_response_rate:** Porcentaje que indica la frecuencia con la que el anfitrión responde a las consultas de los huéspedes.
- **host_acceptance_rate:** Porcentaje que indica la frecuencia con la que el anfitrión acepta solicitudes de reserva.
- **host_is_superhost:** Indicador (generalmente booleano) de si el anfitrión es considerado un "Superhost", lo cual es un título que Airbnb otorga a anfitriones que cumplen con ciertos criterios de calidad y rendimiento.
- **host_total_listings_count:** Número total de anuncios que el anfitrión tiene en Airbnb.
- **neighbourhood:** Barrio donde se encuentra el anuncio.
- **city:** Ciudad donde se encuentra el anuncio.
- **latitude:** Latitud geográfica de la propiedad.
- **longitude:** Longitud geográfica de la propiedad.
- **property_type:** Tipo de propiedad del anuncio (por ejemplo, apartamento, casa, habitación privada, etc.).
- **accommodates:** Número de personas que el anuncio puede alojar.
- **bedrooms:** Número de habitaciones disponibles en el anuncio.
- **price:** Precio por noche del anuncio.
- **review_scores_rating:** Calificación promedio del anuncio basada en las reseñas de los huéspedes.
- **instant_bookable:** Indicador de si el anuncio puede ser reservado inmediatamente sin necesidad de aprobación previa por parte del anfitrión.
- **listing_size_sqft:** Tamaño de la propiedad del anuncio en pies cuadrados.

In [22]:
df.rename(columns={
    'listing_id': 'ID del anuncio',
    'host_id': 'ID del anfitrión',
    'host_since_datekey': 'Fecha de inicio del anfitrión',
    'host_response_rate': 'Tasa de respuesta del anfitrión',
    'host_acceptance_rate': 'Tasa de aceptación del anfitrión',
    'host_is_superhost': 'Es Superanfitrión',
    'host_total_listings_count': 'Total de anuncios del anfitrión',
    'neighbourhood': 'Barrio',
    'city': 'Ciudad',
    'latitude': 'Latitud',
    'longitude': 'Longitud',
    'property_type': 'Tipo de propiedad',
    'accommodates': 'Capacidad',
    'bedrooms': 'Habitaciones',
    'price': 'Precio',
    'review_scores_rating': 'Calificación de reseñas',
    'instant_bookable': 'Reserva inmediata',
    'listing_size_sqft': 'Tamaño del anuncio en pies cuadrados'
}, inplace=True)


In [23]:
df

,ID del anuncio,ID del anfitrión,Fecha de inicio del anfitrión,Tasa de respuesta del anfitrión,Tasa de aceptación del anfitrión,Es Superanfitrión,Total de anuncios del anfitrión,Barrio,Ciudad,Latitud,Longitud,Tipo de propiedad,Capacidad,Habitaciones,Precio,Calificación de reseñas,Reserva inmediata,Tamaño del anuncio en pies cuadrados
0,46096773,349135088,20200608,NaN,0.26,f,5.0,Enclos-St-Laurent,Paris,48.879289,2.354539,Room in hotel,0,NaN,0.0,18,f,130.565530
1,44692017,361914459,20200812,NaN,0.34,f,0.0,Palais-Bourbon,Paris,48.854270,2.307810,Room in boutique hotel,0,NaN,0.0,88,f,286.832585
2,45841514,371644174,20201012,NaN,0.51,f,0.0,Murray Hill,New York,40.750920,-73.980050,Room in boutique hotel,0,NaN,0.0,75,f,294.833871
3,42228997,314197504,20191203,NaN,0.33,f,0.0,Lower East Side,New York,40.721860,-73.992780,Room in hotel,4,NaN,0.0,13,f,403.228927
4,48115028,387570084,20200429,NaN,0.64,f,0.0,Murray Hill,New York,40.750920,-73.980050,Room in boutique hotel,0,NaN,0.0,92,f,342.782170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7205,48109312,387561857,20200526,1.0,0.47,f,0.0,Fremont,Sydney,-33.889040,151.270220,NaN,6,3.0,NaN,96,f,NaN
7206,6569075,235767408,20190110,NaN,0.61,f,19.0,Fremont,Sydney,-33.893720,151.216610,NaN,2,1.0,NaN,96,t,NaN
7207,7736378,295593456,20190917,1.0,0.34,t,2.0,Fremont,Sydney,-33.880730,151.213870,NaN,2,1.0,NaN,96,t,NaN
7208,7699700,293573248,20190910,1.0,0.91,f,0.0,Fremont,Sydney,-33.875050,151.221260,NaN,2,1.0,NaN,98,t,NaN


In [24]:
df["Fecha de inicio del anfitrión"]=pd.to_datetime(  df["Fecha de inicio del anfitrión"], format="%Y%m%d")

In [6]:
df

,ID del anuncio,ID del anfitrión,Fecha de inicio del anfitrión,Tasa de respuesta del anfitrión,Tasa de aceptación del anfitrión,Es Superanfitrión,Total de anuncios del anfitrión,Barrio,Ciudad,Latitud,Longitud,Tipo de propiedad,Capacidad,Habitaciones,Precio,Calificación de reseñas,Reserva inmediata,Tamaño del anuncio en pies cuadrados
0,46096773,349135088,2020-06-08,NaN,0.26,f,5.0,Enclos-St-Laurent,Paris,48.879289,2.354539,Room in hotel,0,NaN,0.0,18,f,130.565530
1,44692017,361914459,2020-08-12,NaN,0.34,f,0.0,Palais-Bourbon,Paris,48.854270,2.307810,Room in boutique hotel,0,NaN,0.0,88,f,286.832585
2,45841514,371644174,2020-10-12,NaN,0.51,f,0.0,Murray Hill,New York,40.750920,-73.980050,Room in boutique hotel,0,NaN,0.0,75,f,294.833871
3,42228997,314197504,2019-12-03,NaN,0.33,f,0.0,Lower East Side,New York,40.721860,-73.992780,Room in hotel,4,NaN,0.0,13,f,403.228927
4,48115028,387570084,2020-04-29,NaN,0.64,f,0.0,Murray Hill,New York,40.750920,-73.980050,Room in boutique hotel,0,NaN,0.0,92,f,342.782170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7205,48109312,387561857,2020-05-26,1.0,0.47,f,0.0,Fremont,Sydney,-33.889040,151.270220,NaN,6,3.0,NaN,96,f,NaN
7206,6569075,235767408,2019-01-10,NaN,0.61,f,19.0,Fremont,Sydney,-33.893720,151.216610,NaN,2,1.0,NaN,96,t,NaN
7207,7736378,295593456,2019-09-17,1.0,0.34,t,2.0,Fremont,Sydney,-33.880730,151.213870,NaN,2,1.0,NaN,96,t,NaN
7208,7699700,293573248,2019-09-10,1.0,0.91,f,0.0,Fremont,Sydney,-33.875050,151.221260,NaN,2,1.0,NaN,98,t,NaN


In [25]:
df.shape

(7210, 18)

In [26]:
df.columns

Index(['ID del anuncio', 'ID del anfitrión', 'Fecha de inicio del anfitrión',
       'Tasa de respuesta del anfitrión', 'Tasa de aceptación del anfitrión',
       'Es Superanfitrión', 'Total de anuncios del anfitrión', 'Barrio',
       'Ciudad', 'Latitud', 'Longitud', 'Tipo de propiedad', 'Capacidad',
       'Habitaciones', 'Precio', 'Calificación de reseñas',
       'Reserva inmediata', 'Tamaño del anuncio en pies cuadrados'],
      dtype='object')

In [27]:
df["Tipo de propiedad"].unique()

array(['Room in hotel', 'Room in boutique hotel', 'Entire apartment',
       'Private room in apartment', 'Shared room in apartment',
       'Shared room in guest suite', 'Room in bed and breakfast',
       'Room in hostel', 'Shared room in townhouse',
       'Shared room in hostel', 'Shared room in bed and breakfast',
       'Private room in villa', 'Private room in condominium',
       'Shared room in serviced apartment', 'Shared room in house',
       'Private room in townhouse', 'Private room in house',
       'Private room in loft', 'Private room in bed and breakfast',
       'Private room in guest suite', 'Entire condominium',
       'Private room in guesthouse', 'Entire serviced apartment',
       'Shared room in guesthouse', 'Private room in tiny house',
       'Shared room in condominium', 'Room in serviced apartment',
       'Entire chalet', 'Private room in bungalow', 'Entire guest suite',
       'Private room in serviced apartment', 'Entire house',
       'Entire loft', 'Sh

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7210 entries, 0 to 7209
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   ID del anuncio                        7210 non-null   int64         
 1   ID del anfitrión                      7210 non-null   int64         
 2   Fecha de inicio del anfitrión         7210 non-null   datetime64[ns]
 3   Tasa de respuesta del anfitrión       4153 non-null   float64       
 4   Tasa de aceptación del anfitrión      7210 non-null   float64       
 5   Es Superanfitrión                     7210 non-null   object        
 6   Total de anuncios del anfitrión       7202 non-null   float64       
 7   Barrio                                7210 non-null   object        
 8   Ciudad                                7210 non-null   object        
 9   Latitud                               7210 non-null   float64       
 10  

In [11]:
df.describe()

,ID del anuncio,ID del anfitrión,Fecha de inicio del anfitrión,Tasa de respuesta del anfitrión,Tasa de aceptación del anfitrión,Total de anuncios del anfitrión,Latitud,Longitud,Capacidad,Habitaciones,Precio,Calificación de reseñas,Tamaño del anuncio en pies cuadrados
count,7.210000e+03,7.210000e+03,7210,4153.000000,7210.000000,7202.000000,7210.000000,7210.000000,7210.000000,6413.000000,7196.000000,7210.000000,7196.000000
mean,4.159495e+07,3.161178e+08,2019-12-14 17:00:22.968100096,0.846017,0.582302,12.316301,20.834321,25.786188,3.182663,1.431779,149.760839,75.123440,873.594483
min,1.245200e+04,1.336740e+07,2019-01-01 00:00:00,0.000000,0.250000,0.000000,-34.088190,-74.166070,0.000000,1.000000,0.000000,-50.000000,83.344224
25%,3.797416e+07,2.712750e+08,2019-06-25 00:00:00,0.890000,0.420000,0.000000,-33.789310,-73.889290,2.000000,1.000000,55.000000,59.000000,395.643324
50%,4.228178e+07,3.113271e+08,2019-11-22 00:00:00,1.000000,0.570000,1.000000,40.757655,12.267095,2.000000,1.000000,87.000000,85.000000,616.162973
75%,4.647144e+07,3.649954e+08,2020-06-14 18:00:00,1.000000,0.720000,3.000000,41.908288,151.081298,4.000000,2.000000,143.000000,94.000000,993.194325
max,4.811632e+07,3.875705e+08,2021-02-06 00:00:00,1.000000,0.990000,1813.000000,48.904720,151.337320,16.000000,11.000000,10571.000000,160.000000,27246.400430
std,5.713025e+06,5.004174e+07,NaN,0.301183,0.197114,110.517077,35.309111,87.464063,2.136565,0.856829,384.286815,24.962839,1157.617799


In [12]:
df.describe(include="object")

,Es Superanfitrión,Barrio,Ciudad,Tipo de propiedad,Reserva inmediata
count,7210,7210,7210,7196,7210
unique,2,230,4,62,2
top,f,I Centro Storico,Sydney,Entire apartment,t
freq,6215,847,2110,3062,3945


In [13]:
df.isnull().sum()

ID del anuncio                             0
ID del anfitrión                           0
Fecha de inicio del anfitrión              0
Tasa de respuesta del anfitrión         3057
Tasa de aceptación del anfitrión           0
Es Superanfitrión                          0
Total de anuncios del anfitrión            8
Barrio                                     0
Ciudad                                     0
Latitud                                    0
Longitud                                   0
Tipo de propiedad                         14
Capacidad                                  0
Habitaciones                             797
Precio                                    14
Calificación de reseñas                    0
Reserva inmediata                          0
Tamaño del anuncio en pies cuadrados      14
dtype: int64

#### Pasos de preprocesamiento de datos
Antes de sumergirnos en PCA, debemos asegurarnos de que nuestros datos estén limpios y en el formato correcto para el análisis:

- **Valores faltantes:** manejamos los valores faltantes rellenándolos con la media de sus respectivas columnas, asegurándonos de que no se dejara ningún punto de datos.

- **Codificación categórica :** convertimos variables categóricas como host_is_superhost,,, y en numéricas usando codificación de etiquetas, mientras que la característica estaba codificada en caliente. Este paso es crucial ya que PCA requiere entrada numérica.  `neighbourhood`  `property_type` `instant_bookablecity`
- **Escalado:**  usamos `StandardScalerescalar`. El escalado es vital para PCA porque es sensible a las variaciones de las variables iniciales.

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7210 entries, 0 to 7209
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   ID del anuncio                        7210 non-null   int64         
 1   ID del anfitrión                      7210 non-null   int64         
 2   Fecha de inicio del anfitrión         7210 non-null   datetime64[ns]
 3   Tasa de respuesta del anfitrión       4153 non-null   float64       
 4   Tasa de aceptación del anfitrión      7210 non-null   float64       
 5   Es Superanfitrión                     7210 non-null   object        
 6   Total de anuncios del anfitrión       7202 non-null   float64       
 7   Barrio                                7210 non-null   object        
 8   Ciudad                                7210 non-null   object        
 9   Latitud                               7210 non-null   float64       
 10  

In [15]:
df["Tipo de propiedad"]

0                Room in hotel
1       Room in boutique hotel
2       Room in boutique hotel
3                Room in hotel
4       Room in boutique hotel
                 ...          
7205                       NaN
7206                       NaN
7207                       NaN
7208                       NaN
7209                       NaN
Name: Tipo de propiedad, Length: 7210, dtype: object

In [16]:
import numpy as np

# Selecciona solo las columnas numéricas para calcular la media
mean_values = df.select_dtypes(include=[np.number]).mean()

In [17]:
# Calcular la moda de la columna categórica
moda = df["Tipo de propiedad"].mode()[0]

# Rellenar los valores faltantes en la columna categórica con la moda
df["Tipo de propiedad"] = df["Tipo de propiedad"].fillna(moda)

In [18]:
df = df.fillna(mean_values) 

In [19]:


# Convierta columnas categóricas a numéricas usando codificación de etiquetas 
# Inicialice el codificador de etiquetas
label_encoder = LabelEncoder() 

# Columnas para codificar etiquetas
label_encode_columns = ['Es Superanfitrión', 'Barrio', 'Tipo de propiedad', 'Reserva inmediata'] 

# Aplicar codificación de etiqueta a cada columna
for columna in label_encode_columns:
    df[columna] = label_encoder.fit_transform(df[columna]) 

# Aplicar codificación one-hot a 'ciudad' usando get_dummies
df = pd.get_dummies(df, columns=['Ciudad']) 

# Redefinir y reajustar el escalador al conjunto de datos actual
scaler = StandardScaler() 
scaled_features = scaler.fit_transform(df)

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'nump